<a href="https://colab.research.google.com/github/Lawrence-Krukrubo/Building-a-Forward-Planning-Agent/blob/master/forward_planning_agent_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
!rm planning.xlsx
!wget https://storage.googleapis.com/law1010/AIND_project_3_experiment.xlsx -O planning.xlsx

rm: cannot remove 'planning.xlsx': No such file or directory
--2020-09-17 09:12:14--  https://storage.googleapis.com/law1010/AIND_project_3_experiment.xlsx
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11205 (11K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘planning.xlsx’

planning.xlsx       100%[===================>]  10.94K  --.-KB/s    in 0s      

2020-09-17 09:12:15 (34.1 MB/s) - ‘planning.xlsx’ saved [11205/11205]



In [5]:
xls = pd.ExcelFile('planning.xlsx')
print(type(xls))

<class 'pandas.io.excel._base.ExcelFile'>


Let's see the sheet names

In [6]:
print(xls.sheet_names)

['problem_1', 'problem_2', 'problem _3', 'problem_4']


In [7]:
prob_1 = pd.read_excel('planning.xlsx', sheet_name='problem_1', skiprows=2)
prob_1

,Problem,Algorithm,Actions,Expansions,Goal_tests,New_nodes,Plan_length,Time
0,Air Cargo Problem 1,breadth_first_search,20,43,56,178,6,0.004324
1,Air Cargo Problem 1,depth_first_graph_search,20,21,22,84,20,0.002299
2,Air Cargo Problem 1,uniform_cost_search,20,60,62,240,6,0.006638
3,Air Cargo Problem 1,greedy_best_first_graph_search h_unmet_goals,20,7,9,29,6,0.001128
4,Air Cargo Problem 1,greedy_best_first_graph_search h_pg_levelsum,20,6,8,28,6,0.076109
5,Air Cargo Problem 1,greedy_best_first_graph_search h_pg_maxlevel,20,6,8,24,6,0.053792
6,Air Cargo Problem 1,greedy_best_first_graph_search h_pg_setlevel,20,6,8,28,6,0.261131
7,Air Cargo Problem 1,astar_search h_unmet_goals,20,50,52,206,6,0.006688
8,Air Cargo Problem 1,astar_search h_pg_levelsum,20,28,30,122,6,0.186529
9,Air Cargo Problem 1,astar_search h_pg_maxlevel,20,43,45,180,6,0.198359


For Problem 2:

In [ ]:
prob_2 = pd.read_excel('planning.xlsx', sheet_name='problem_2', skiprows=2)
prob_2

,Problem,Algorithm,Actions,Expansions,Goal_tests,New_nodes,Plan_length,Time
0,Air Cargo Problem 2,breadth_first_search,72,3343,4609,30503,9,1.412719
1,Air Cargo Problem 2,depth_first_graph_search,72,624,625,5602,619,1.813666
2,Air Cargo Problem 2,uniform_cost_search,72,5154,5156,46618,9,2.400566
3,Air Cargo Problem 2,greedy_best_first_graph_search h_unmet_goals,72,17,19,170,9,0.014487
4,Air Cargo Problem 2,greedy_best_first_graph_search h_pg_levelsum,72,9,11,86,9,1.578095
5,Air Cargo Problem 2,greedy_best_first_graph_search h_pg_maxlevel,72,27,29,249,9,2.507638
6,Air Cargo Problem 2,greedy_best_first_graph_search h_pg_setlevel,72,9,11,84,9,6.143889
7,Air Cargo Problem 2,astar_search h_unmet_goals,72,2467,2469,22522,9,1.584434
8,Air Cargo Problem 2,astar_search h_pg_levelsum,72,357,359,3426,9,65.294872
9,Air Cargo Problem 2,astar_search h_pg_maxlevel,72,2887,2889,26594,9,261.383882


For problem 3

In [ ]:
prob_3 = pd.read_excel('planning.xlsx', sheet_name='problem _3', skiprows=2)
prob_3

,Problem,Algorithm,Actions,Expansions,Goal_tests,New_nodes,Plan_length,Time
0,Air Cargo Problem 3,breadth_first_search,88,14663,18098,129625,12,7.490619
1,Air Cargo Problem 3,depth_first_graph_search,88,408,409,3364,392,0.894157
2,Air Cargo Problem 3,greedy_best_first_graph_search h_unmet_goals,88,25,27,230,15,0.030409
3,Air Cargo Problem 3,greedy_best_first_graph_search h_pg_levelsum,88,14,16,126,14,5.022938
4,Air Cargo Problem 3,astar_search h_unmet_goals,88,7388,7390,65711,12,9.222211
5,Air Cargo Problem 3,astar_search h_pg_levelsum,88,369,371,3403,12,93.623633


For problem 4

In [ ]:
prob_4 = pd.read_excel('planning.xlsx', sheet_name='problem_4', skiprows=2)
prob_4

,Problem,Algorithm,Actions,Expansions,Goal_tests,New_nodes,Plan_length,Time
0,Air Cargo Problem 4,breadth_first_search,104,99736,114953,944130,14,72.864881
1,Air Cargo Problem 4,depth_first_graph_search,104,25174,25175,228849,24132,2465.383703
2,Air Cargo Problem 4,greedy_best_first_graph_search h_unmet_goals,104,29,31,280,18,0.037212
3,Air Cargo Problem 4,greedy_best_first_graph_search h_pg_levelsum,104,17,19,165,17,6.834336
4,Air Cargo Problem 4,astar_search h_unmet_goals,104,34330,34332,328509,14,50.453265
5,Air Cargo Problem 4,astar_search h_pg_levelsum,104,1208,1210,12210,15,600.920339
